In [1]:
import geemap
from geemap import colormaps
from geemap import foliumap
import ee
import sys
sys.path.append('C:/Users/wb514197/Repos/GEE_Zonal/src')
from gee_tools import Catalog, ZonalStats
from gee_helpers import get_zonal_res
import pandas as pd
import os
import glob
import seaborn as sns
from datetime import datetime
from matplotlib import pyplot as plt
from geemap.conversion import *
import geopandas as gpd

eemont not available
Google Drive tools are not available


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# ee.Authenticate()

In [4]:
ee.Initialize()

In [5]:
# pac_dir = os.path.join(os.path.expanduser("~"), 'data', 'pacific')
pac_dir = 'X:/data/pacific'
png_dir = os.path.join(pac_dir, "PNG")
out_dir = os.path.join(pac_dir, 'temp')
min_dir = os.path.join(png_dir, 'mining')

In [6]:
# pac_path = 'C:/Users/wb514197/WBG/Pacific Observatory - WB Group - Geospatial and Big Data/Data/Admin Boundaries'
pac_path = r'C:\Users\wb514197\WBG\EEAPV Pacific Observatory Files - Geospatial and Big Data\Data\Admin Boundaries'
pac_adm0 = gpd.read_file(os.path.join(pac_path, "Adm0_Pacific_Edit.shp"))

In [7]:
sel = [
    'Federated States of Micronesia', 
    'Fiji',
    'Kiribati',
    'Marshall Islands',
    'Nauru',
    'Palau',
    'Papua New Guinea',
    'Samoa',
    'Solomon Islands',
    'Tonga',
    'Tuvalu',
    'Vanuatu'
]

In [8]:
pac = pac_adm0.loc[pac_adm0.WB_ADM0_NA.isin(sel)].copy()

In [9]:
viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
    .filterDate('2021-01-01', "2021-12-31")
#     .select('avg_rad') \

In [10]:
def prepareImage(img):
    rad = img.select('avg_rad')
    cf_cvg = img.select('cf_cvg') #.rename('avg_rad')
    cf_eq0 = cf_cvg.gt(0)
    img_masked = rad.mask(cf_eq0)
    
    return img_masked

In [11]:
data = viirs.map(prepareImage)

In [12]:
composite = data.mean()

In [74]:
aoi = pac.loc[pac.WB_ADM0_NA=="Vanuatu"].copy()
aoi.geometry = aoi.buffer(2000)
aoi = aoi.to_crs("EPSG:4326")

In [75]:
aoi = geopandas_to_ee(aoi)

In [76]:
roi = aoi.geometry()

In [77]:
country_comp = composite.clip(roi)
country_comp_masked = composite.updateMask(country_comp)
country_comp_unmasked = country_comp_masked.unmask(-1)

In [78]:
filename = os.path.join(out_dir, 'vanuatu_2021_unmasked.tif')

In [79]:
geemap.ee_export_image(
    country_comp_unmasked, filename=filename, scale=500, region=roi, file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to X:\data\pacific\temp\vanuatu_2021_unmasked.tif


In [91]:
geemap.ee_export_image_to_drive?

In [102]:
for idx, row in pac.iterrows():
    aoi = pac.loc[pac.WB_ADM0_NA==row.WB_ADM0_NA].copy()
    aoi.geometry = aoi.buffer(2000)
    aoi = aoi.to_crs("EPSG:4326")
    aoi = geopandas_to_ee(aoi)
    roi = aoi.geometry()
    country_comp = composite.clip(roi)
    country_comp_masked = composite.updateMask(country_comp)
    country_comp_unmasked = country_comp_masked.unmask(-1)
#     filename = os.path.join(out_dir, f'{row.ISO3}_2021.tif')
    filename = f'{row.ISO3}_2021'
#     geemap.ee_export_image(
#         country_comp_unmasked, filename=filename, scale=500, region=roi, file_per_band=False
#     )
    geemap.ee_export_image_to_drive(
        country_comp_unmasked, description=filename, folder='pacific-ntl', region=roi, scale=500, maxPixels=1e13
    )
    print(f'done with {row.WB_ADM0_NA}')

done with Fiji
done with Kiribati
done with Marshall Islands
done with Federated States of Micronesia
done with Nauru
done with Palau
done with Papua New Guinea
done with Samoa
done with Solomon Islands
done with Tonga
done with Tuvalu
done with Vanuatu


In [13]:
aoi = pac.loc[pac.WB_ADM0_NA=="Kiribati"].copy()
aoi = aoi.to_crs("EPSG:3832")
aoi.geometry = aoi.buffer(2000)
# aoi = aoi.to_crs("EPSG:4326")

In [15]:
aoi = geopandas_to_ee(aoi)
roi = aoi.geometry()

In [16]:
country_comp = composite.clip(roi)
country_comp_masked = composite.updateMask(country_comp)
country_comp_unmasked = country_comp_masked.unmask(-1)

In [18]:
filename = f'KIR_2021_v2'

In [19]:
country_comp_unmasked.projection().getInfo()

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}

In [20]:
geemap.ee_export_image_to_drive(
    country_comp_unmasked, description=filename, folder='pacific-ntl', region=roi, scale=500, maxPixels=1e13, crs='EPSG:3832'
)